In [ ]:
import torch

from rnampnn.utils.data import analyse_dataset

analyse_dataset()

In [ ]:
from rnampnn.utils.data import RNADataModule

data = RNADataModule()
data.setup(stage="fit")

In [ ]:
train = data.train_dataloader()
for i in train:
    print(i)
    break

In [ ]:
from rnampnn.utils.data import RNADataset

dataset = RNADataset.from_path("data/")


In [ ]:
print(len(dataset))
dataset.slice_augmentation(num_gen=100)
print(len(dataset))

In [ ]:
import torch
def dist(X, mask, eps=1E-6):
    mask_2D = torch.unsqueeze(mask, 1) * torch.unsqueeze(mask, 2)
    dX = torch.unsqueeze(X, 1) - torch.unsqueeze(X, 2)
    dX = torch.sum(dX ** 2, 3)
    D = (1. - mask_2D) * 10000 + mask_2D * torch.sqrt(dX + eps)
    print("D:", D)

    D_max, _ = torch.max(D, -1, keepdim=True)
    print("D_max:", D_max.shape)
    D_adjust = D + (1. - mask_2D) * (D_max + 1)
    print("D_adjust:",D_adjust, D_adjust.shape)
    D_neighbors, E_idx = torch.topk(D_adjust, min(5, D_adjust.shape[-1]), dim=-1, largest=False)
    print("D_neighbors:", D_neighbors.shape)
    print("E_idx:", E_idx.shape)
    return D_neighbors, E_idx

X = torch.cat((torch.rand(1, 10, 6, 3),torch.zeros(1,5,6,3)), dim=1)
X = X[:, :, 0, :]
print(X.shape)
mask = torch.cat((torch.ones(1,10), torch.zeros(1,5)), dim=1)
Y = dist(X, mask)
print(Y[0], Y[1])

In [1]:
from rnampnn.model.rnampnn import AtomFeature, AtomMPNN, AtomPooling
import torch

atom_feature = AtomFeature(num_neighbour=30)
X = torch.cat(( torch.rand(1, 32, 7, 3), torch.zeros(1, 4, 7, 3)), dim=1)
mask = torch.cat((torch.ones(1, 32), torch.zeros(1, 4)), dim=1)
print(mask)
atom_coords, atom_mask, encode, dist_neighbors, edge_index = atom_feature(X, mask)
atom_mpnn = AtomMPNN(hidden_dim=32, num_layers=2)
updated_encode, _, _, _ = atom_mpnn(encode, atom_mask, dist_neighbors, edge_index)

raw_feature = torch.cat(( torch.rand(1, 32, 8), torch.zeros(1, 4, 8)), dim=1)
atom_pooling = AtomPooling(raw_feature_dim=8, hidden_dim=32, num_layers=2)
pooling = atom_pooling(updated_encode, atom_mask, raw_feature)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.]])
torch.Size([1, 36])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
      